# 05: Extracción de Boletines

**Propósito:** Este *notebook* extrae los metadatos y el **texto completo** de los boletines (proyectos de ley) asociados a las votaciones de cada período.

**Lógica:**
1.  Itera por cada período.
2.  Lee el `detalle.csv` (de `04_Extraction_Votes`) para encontrar los IDs de boletín únicos (ej. "2625-07").
3.  Llama a la función `build_boletines_periodo` (de `src/bulletin_utils.py`).
4.  Esa función orquesta la extracción:
    a. Llama a la API XML del Senado (`tramitacion.senado.cl`) para cada ID de boletín.
    b. Parsea el XML para obtener metadatos (título, autores, materias, `link_mensaje_mocion`).
    c. Scrapea el `link_mensaje_mocion` para descargar el **texto completo** del proyecto.
5.  Guarda el DataFrame resultante.

**Dependencias:**
* `data/01_raw/periodos_master.csv`
* `data/01_raw/[periodo]/detalle.csv`

**Salidas (Artifacts):**
* `data/01_raw/[periodo]/boletines.csv` (Contiene el `texto_completo` para NLI)

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm 

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    # Asumimos que 'sanitize_filename' está en un módulo común
    from src.common_utils import sanitize_filename, get_ollama_client
    # Importamos la función orquestadora principal de nuestro NUEVO módulo
    from src.bulletin_utils import build_boletines_periodo 
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    logging.error("Asegúrese de que 'src/common_utils.py' y 'src/bulletin_utils.py' existan.")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# --- Definir el ARCHIVO DE ENTRADA (Dependencia) ---
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")

2025-10-26 18:05:08,101 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-26 18:05:08,104 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw


## 2. Carga de Dependencias (Períodos)

Cargamos la lista maestra de períodos sobre la cual vamos a iterar.

In [3]:
try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
    display(df_periodos.head())
except FileNotFoundError as e:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {MASTER_PERIOD_FILE}")
    logging.error("Por favor, ejecute el notebook '00_Extraction_Periods.ipynb' primero.")
    raise

2025-10-26 18:05:08,115 - INFO - Se cargó la lista maestra de 10 períodos.


,Id,Nombre,FechaInicio,FechaTermino
0,7,1965-1969,1965-03-11,1969-03-11 00:00:00
1,1,1990-1994,1990-03-11,1994-03-10 00:00:00
2,2,1994-1998,1994-03-11,1998-03-10 00:00:00
3,3,1998-2002,1998-03-11,2002-03-10 00:00:00
4,4,2002-2006,2002-03-11,2006-03-10 00:00:00


## 3. Bucle Principal de Extracción de Boletines

Iteramos sobre cada período. La función `build_boletines_periodo` (en `/src`) se encarga de todo el trabajo pesado, incluyendo mostrar una barra de progreso anidada para la descarga de boletines.

In [4]:
logging.info(f"Iniciando extracción de boletines para {len(df_periodos)} períodos...")

for row in tqdm(df_periodos.itertuples(), total=len(df_periodos), desc="Procesando Períodos"):
    
    nombre_periodo = row.Nombre
    nombre_carpeta = sanitize_filename(nombre_periodo)
    carpeta_periodo = DATA_DIR_RAW / nombre_carpeta
    
    # --- Definir rutas de Input y Output ---
    ruta_output_boletines = carpeta_periodo / "boletines.csv"

    logging.info(f"--- Procesando Período: {nombre_periodo} ---")

    if ruta_output_boletines.exists():
        logging.info(f"Saltando período: El archivo 'boletines.csv' ya existe.")
        continue

    try:
        logging.info("Llamando a build_boletines_periodo...")
        df_boletines = build_boletines_periodo(nombre_periodo, DATA_DIR_RAW)
        
        if df_boletines is None or df_boletines.empty:
            logging.warning(f"No se generaron datos de boletines para el período {nombre_periodo}.")
            continue

        carpeta_periodo.mkdir(parents=True, exist_ok=True)

        df_boletines.to_csv(ruta_output_boletines, index=False, encoding="utf-8")
        logging.info(f"Guardado exitosamente: {ruta_output_boletines} ({len(df_boletines)} filas)")

    except Exception as e:
        logging.error(f"ERROR FATAL al procesar período {nombre_periodo}: {e}", exc_info=True)

logging.info("--- Extracción de boletines finalizada ---")

2025-10-26 18:05:08,134 - INFO - Iniciando extracción de boletines para 10 períodos...


Procesando Períodos:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-26 18:05:08,162 - INFO - --- Procesando Período: 1965-1969 ---
2025-10-26 18:05:08,163 - INFO - Llamando a build_boletines_periodo...
2025-10-26 18:05:08,164 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1965-1969\detalle.csv.
2025-10-26 18:05:08,165 - WARNING - No se generaron datos de boletines para el período 1965-1969.
2025-10-26 18:05:08,165 - INFO - --- Procesando Período: 1990-1994 ---
2025-10-26 18:05:08,166 - INFO - Llamando a build_boletines_periodo...
2025-10-26 18:05:08,167 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1990-1994\detalle.csv.
2025-10-26 18:05:08,168 - WARNING - No se generaron datos de boletines para el período 1990-1994.
2025-10-26 18:05:08,168 - INFO - --- Procesando Período: 1994-1998 ---
2025-10-26 18:05:08,169 - INFO - Llamando a build_boletines_periodo...
2025

Boletines 2002-2006:   0%|          | 0/476 [00:00<?, ?it/s]

2025-10-26 18:05:22,621 - ERROR - Error de parseo en get_boletin (XML) para 3116: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\notebooks\..\src\bulletin_utils.py", line 35, in get_boletin
    proyecto = data_dict.get('proyectos', {}).get('proyecto')
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'
2025-10-26 18:05:42,899 - ERROR - Error de parseo en get_boletin (XML) para 2930: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\notebooks\..\src\bulletin_utils.py", line 35, in get_boletin
    proyecto = data_dict.get('proyectos', {}).get('proyecto')
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'
2025-1

...

2025-10-26 18:33:40,778 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2014-2018\boletines.csv (572 filas)
2025-10-26 18:33:40,779 - INFO - --- Procesando Período: 2018-2022 ---
2025-10-26 18:33:40,780 - INFO - Llamando a build_boletines_periodo...
2025-10-26 18:33:40,782 - INFO - Cargando C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2018-2022\detalle.csv para extraer IDs de boletín.
2025-10-26 18:33:45,310 - INFO - Encontrados 699 boletines únicos. Iniciando descarga...


Boletines 2018-2022:   0%|          | 0/699 [00:00<?, ?it/s]

2025-10-26 18:43:10,168 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2018-2022\boletines.csv (699 filas)
2025-10-26 18:43:10,170 - INFO - --- Procesando Período: 2022-2026 ---
2025-10-26 18:43:10,171 - INFO - Llamando a build_boletines_periodo...
2025-10-26 18:43:10,172 - INFO - Cargando C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2022-2026\detalle.csv para extraer IDs de boletín.
2025-10-26 18:43:15,091 - INFO - Encontrados 667 boletines únicos. Iniciando descarga...


Boletines 2022-2026:   0%|          | 0/667 [00:00<?, ?it/s]

2025-10-26 18:51:59,108 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2022-2026\boletines.csv (667 filas)
2025-10-26 18:51:59,110 - INFO - --- Extracción de boletines finalizada ---
